Dataset Description
-------------------

In this competition your task is to predict whether a passenger was transported to an alternate dimension during the *Spaceship Titanic*'s collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.

File and Data Field Descriptions
================================

-   **train.csv** - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
    -   `PassengerId` - A unique Id for each passenger. Each Id takes the form `gggg_pp` where `gggg` indicates a group the passenger is travelling with and `pp` is their number within the group. People in a group are often family members, but not always.
    -   `HomePlanet` - The planet the passenger departed from, typically their planet of permanent residence.
    -   `CryoSleep` - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
    -   `Cabin` - The cabin number where the passenger is staying. Takes the form `deck/num/side`, where `side` can be either `P` for *Port* or `S` for *Starboard*.
    -   `Destination` - The planet the passenger will be debarking to.
    -   `Age` - The age of the passenger.
    -   `VIP` - Whether the passenger has paid for special VIP service during the voyage.
    -   `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` - Amount the passenger has billed at each of the *Spaceship Titanic*'s many luxury amenities.
    -   `Name` - The first and last names of the passenger.
    -   `Transported` - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.
-   **test.csv** - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of `Transported` for the passengers in this set.
-   **sample_submission.csv** - A submission file in the correct format.
    -   `PassengerId` - Id for each passenger in the test set.
    -   `Transported` - The target. For each passenger, predict either `True` or `False`.

## EDA

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn


In [3]:
df = pd.read_csv("data/train.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
df.Destination.unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

## Data Prep and Feature Engineering

In [5]:
df['Group'] = df['PassengerId'].str.split('_').str[0]
df['n_group'] = df.groupby(['Group'])['Group'].transform('count')

In [6]:
# split cabin features
df[["deck","num","side"]] = df['Cabin'].str.split("/", expand=True)
df['num'] = df['num'].astype('Int64')

df['deck'].unique(), df['side'].unique(), df['num'].unique()


(array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object),
 array(['P', 'S', nan], dtype=object),
 <IntegerArray>
 [   0,    1,    2,    3, <NA>,    4,    5,    6,    8,    9,
  ...
  1782, 1884,  353, 1886, 1498, 1888, 1891, 1892, 1893, 1894]
 Length: 1818, dtype: Int64)

In [7]:
# fix wrong country
df['HomePlanet'] = df['HomePlanet'].replace("Europa", "Earth")

df['HomePlanet'].unique()

array(['Earth', 'Mars', nan], dtype=object)

In [8]:
# convert booleans
df['CryoSleep'] = df['CryoSleep']*1
df['VIP'] = df['VIP']*1

In [9]:
features = pd.Index([
    # 'PassengerId',
    #'Cabin',
    # 'Name',
    'HomePlanet',
    'Destination',
    'side',#binary
    'deck',
    #'Group',

    'CryoSleep',
    'VIP',
    
    'Age',
    'RoomService',
    'FoodCourt',
    'ShoppingMall',
    'Spa',
    'VRDeck',
    
    
    'n_group',
    #'num',
    


    # 'Transported', # TARGET
])



dffeat = df[features].convert_dtypes()

dffeat = dffeat.replace(np.nan, pd.NA)

cat_features = dffeat.select_dtypes(include='string').columns
num_features = dffeat.select_dtypes(exclude='string').columns

print(f"{cat_features=}\n{num_features=}")

dffeat.info()
dffeat.head(3)

cat_features=Index(['HomePlanet', 'Destination', 'side', 'deck'], dtype='object')
num_features=Index(['CryoSleep', 'VIP', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'n_group'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8492 non-null   string
 1   Destination   8511 non-null   string
 2   side          8494 non-null   string
 3   deck          8494 non-null   string
 4   CryoSleep     8476 non-null   Int64 
 5   VIP           8490 non-null   Int64 
 6   Age           8514 non-null   Int64 
 7   RoomService   8512 non-null   Int64 
 8   FoodCourt     8510 non-null   Int64 
 9   ShoppingMall  8485 non-null   Int64 
 10  Spa           8510 non-null   Int64 
 11  VRDeck        8505 non-null   Int64 
 12  n_group       8693 non-null   Int64 
dtypes: Int64(9), string(4)
memo

,HomePlanet,Destination,side,deck,CryoSleep,VIP,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,n_group
0,Earth,TRAPPIST-1e,P,B,0,0,39,0,0,0,0,0,1
1,Earth,TRAPPIST-1e,S,F,0,0,24,109,9,25,549,44,1
2,Earth,TRAPPIST-1e,S,A,0,1,58,43,3576,0,6715,49,2


In [10]:
# pipe:
# - imputar dados faltantes
# - fazer encoding das variaveis categoricas
# - padronizar/normalizar dados?
# - 

In [11]:
from sklearn.metrics import get_scorer_names
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, KFold

#get_scorer_names()

In [12]:
X = dffeat
y = df["Transported"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
cat_prep = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent", missing_values=pd.NA)),
    ("oh", OneHotEncoder())
])

num_prep = SimpleImputer(strategy="median", missing_values=pd.NA)

prep = ColumnTransformer(
    [
        ("cat", cat_prep, cat_features),
        ("num", num_prep, num_features),
    ],
    verbose_feature_names_out =False
)

prep#.fit_transform(X)
#X = pd.DataFrame(transf.fit_transform(X), index=X.index, columns=transf.get_feature_names_out())

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imp',
                                                  SimpleImputer(missing_values=<NA>,
                                                                strategy='most_frequent')),
                                                 ('oh', OneHotEncoder())]),
                                 Index(['HomePlanet', 'Destination', 'side', 'deck'], dtype='object')),
                                ('num',
                                 SimpleImputer(missing_values=<NA>,
                                               strategy='median'),
                                 Index(['CryoSleep', 'VIP', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'n_group'],
      dtype='object'))],
                  verbose_feature_names_out=False)

In [14]:
pipe = Pipeline([
    ("prep", prep),
    ("model", None)
])

pipe.get_params()

{'memory': None,
 'steps': [('prep', ColumnTransformer(transformers=[('cat',
                                    Pipeline(steps=[('imp',
                                                     SimpleImputer(missing_values=<NA>,
                                                                   strategy='most_frequent')),
                                                    ('oh', OneHotEncoder())]),
                                    Index(['HomePlanet', 'Destination', 'side', 'deck'], dtype='object')),
                                   ('num',
                                    SimpleImputer(missing_values=<NA>,
                                                  strategy='median'),
                                    Index(['CryoSleep', 'VIP', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall',
          'Spa', 'VRDeck', 'n_group'],
         dtype='object'))],
                     verbose_feature_names_out=False)),
  ('model', None)],
 'verbose': False,
 'prep': ColumnTransformer(transfo

In [38]:

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


param_grid = [
    {
        "model": [RandomForestClassifier()],
        # #"model__n_estimators": 100, 
        # #"model__criterion": "gini", 
        # "model__max_depth": [2,8,12,16], 
        # "model__min_samples_split": [2,4,8,16,32], 
        # #"model__min_samples_leaf": 1, 
        # #"model__min_weight_fraction_leaf": 0, 
        # "model__max_features": ["sqrt", "log2", 2,4,6], 
        # "model__max_leaf_nodes": [2,4,8,16], 
        # #"model__min_impurity_decrease": 0, 
        # #"model__bootstrap": True, 
        # #"model__oob_score": False, 
        # #"model__n_jobs": None, 
        # #"model__random_state": None, 
        # #"model__verbose": 0, 
        # #"model__warm_start": False, 
        # #"model__class_weight": None, 
        "model__ccp_alpha": np.linspace(0.001,0.05, num=10), 
        # #"model__max_samples": None
    },
    {
        "model": [GradientBoostingClassifier()],
        "model__learning_rate": np.logspace(-3,0, num=5, endpoint=False),
    },
]


gs = GridSearchCV(pipe, param_grid, scoring=["roc_auc", "accuracy"], refit="roc_auc" ,cv=9, verbose=2, n_jobs=-1).fit(X_train, y_train)


Fitting 9 folds for each of 15 candidates, totalling 135 fits


In [62]:
score = 'roc_auc'
dfres = pd.DataFrame(gs.cv_results_).sort_values(f'rank_test_{score}')
dfres = dfres.rename(columns= lambda x: x.replace('param_model__', '_'))

select = lambda x, pat: x.loc[:,x.columns.str.contains(pat)]

scores = dfres.set_index('params').pipe(select, "mean_test_").head(10)
scores


,mean_test_roc_auc,mean_test_accuracy
params,,
"{'model': GradientBoostingClassifier(learning_rate=0.25118864315095796), 'model__learning_rate': 0.25118864315095796}",0.887332,0.797205
"{'model': GradientBoostingClassifier(learning_rate=0.25118864315095796), 'model__learning_rate': 0.0630957344480193}",0.884149,0.798519
"{'model': RandomForestClassifier(), 'model__ccp_alpha': 0.001}",0.875844,0.793754
"{'model': GradientBoostingClassifier(learning_rate=0.25118864315095796), 'model__learning_rate': 0.015848931924611134}",0.857094,0.784551
"{'model': GradientBoostingClassifier(learning_rate=0.25118864315095796), 'model__learning_rate': 0.003981071705534973}",0.848072,0.758258
"{'model': RandomForestClassifier(), 'model__ccp_alpha': 0.0064444444444444445}",0.847261,0.741002
"{'model': GradientBoostingClassifier(learning_rate=0.25118864315095796), 'model__learning_rate': 0.001}",0.831111,0.730324
"{'model': RandomForestClassifier(), 'model__ccp_alpha': 0.01188888888888889}",0.825987,0.738702
"{'model': RandomForestClassifier(), 'model__ccp_alpha': 0.017333333333333333}",0.804706,0.737223


In [63]:
from sklearn.metrics import roc_auc_score

gs_model = gs.best_estimator_

roc_auc_score(y_train, gs_model.predict(X_train)), roc_auc_score(y_test, gs_model.predict(X_test))

(0.8379280409063037, 0.7943262286159776)

# Predict Test Data